# Installing Missing modules

In [1]:
! pip install wfdb

     |████████████████████████████████| 122kB 5.9MB/s 


# Connecting to Drive

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing Libraries

In [35]:
import numpy as np
import tensorflow as tf
import keras
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


# Reading Data

In [11]:
df = pd.read_pickle('/content/drive/MyDrive/Cardiology/Cardiology_Dataset')
df

,Sequence,Label
0,"[[-127, -162, -197, -229, -245, -254, -261, -2...",0
1,"[[128, 157, 189, 226, 250, 257, 262, 265, 268,...",0
2,"[[56, 73, 85, 93, 100, 107, 113, 117, 118, 117...",0
3,"[[519, 619, 723, 827, 914, 956, 955, 934, 920,...",2
4,"[[-188, -239, -274, -316, -356, -374, -380, -3...",2
...,...,...
8523,"[[-104, -127, -144, -154, -160, -164, -168, -1...",0
8524,"[[-121, -197, -288, -390, -496, -600, -693, -7...",1
8525,"[[-202, -235, -272, -305, -325, -339, -344, -3...",0
8526,"[[-227, -297, -374, -449, -505, -527, -539, -5...",0


# EDA

In [14]:
df['Label'].value_counts()

0    5076
1    2415
2     758
3     279
Name: Label, dtype: int64

In [33]:
fig = px.pie(df, values='Label', names = 'Label', title='Repartition of ECG categories')
fig.show()

## Normal ECG

In [38]:
# To plot

# DataSet Splitting

In [37]:
train, val = train_test_split(df, stratify=df.Label, test_size=0.2)
print(f'Shape of training DataSet : {train.shape}')
print(f'Shape of validation DataSet : {val.shape}')

Shape of training DataSet : (6822, 2)
Shape of validation DataSet : (1706, 2)


In [39]:
X_train = tf.ragged.constant(train.Sequence)
y_train = tf.one_hot(train.Label,4)

X_val = tf.ragged.constant(val.Sequence)
y_val = tf.one_hot(val.Label,4)


In [40]:
print(f'Maximal sequence length of training set : {X_train.bounding_shape()[-1]} ')
print(f'Maximal sequence length of validation set : {X_val.bounding_shape()[-1]} ')

Maximal sequence length of training set : 18286 
Maximal sequence length of validation set : 18264 


# Implementing Model

In [8]:
xx = tf.ragged.constant(df['Sequence'])

yy = tf.one_hot(df['Label'],4)

xx

<tf.RaggedTensor [[[-127, -162, -197, -229, -245, -254, -261, -265, -268, -268, -267, -265, -263, -260, -256, -253, -249, -247, -245, -243, -241, -238, -236, -234, -232, -231, -230, -229, -229, -229, -229, -230, -230, -230, -229, -229, -227, -225, -221, -218, -215, -213, -213, -213, -212, -211, -210, -209, -206, -203, -201, -198, -198, -200, -204, -210, -216, -224, -233, -243, -253, -262, -270, -275, -281, -286, -291, -292, -289, -282, -273, -264, -259, -255, -250, -246, -241, -237, -233, -229, -226, -222, -219, -216, -213, -209, -204, -195, -187, -179, -172, -169, -166, -164, -161, -159, -157, -156, -154, -153, -153, -154, -155, -157, -159, -160, -162, -164, -166, -167, -169, -170, -171, -174, -177, -177, -171, -158, -121, -66, 11, 109, 219, 331, 434, 519, 575, 593, 578, 533, 440, 329, 213, 99, -3, -88, -153, -189, -210, -222, -230, -237, -243, -248, -253, -256, -260, -263, -266, -267, -266, -264, -259, -254, -249, -244, -238, -231, -224, -216, -207, -199, -191, -183, -176, -170, -164

In [41]:
max_seq = X_train.bounding_shape()[-1]
print(f'Maximum length sequence : {max_seq}')

model = keras.Sequential([
                          tf.keras.layers.Input(shape=[None, max_seq], dtype=tf.float32, ragged=True),
                          keras.layers.LSTM(units=64, activation='tanh', dropout=0.2, name = 'LSTM_1', return_sequences=True),
                          keras.layers.LSTM(units=256, activation='tanh', dropout=0.2, name = 'LSTM_2',return_sequences=True),
                          keras.layers.LSTM(units=100, activation='tanh', dropout=0.2, name = 'LSTM_3', return_sequences=False),
                          # tf.keras.layers.LSTM(4),
                          #tf.keras.layers.Flatten(name = 'Flatten'),
                          tf.keras.layers.Dense(units=4, activation='sigmoid', name = 'Dense_1')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

model.summary()

Maximum length sequence : 18286
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM_1 (LSTM)                (None, None, 64)          4697856   
_________________________________________________________________
LSTM_2 (LSTM)                (None, None, 256)         328704    
_________________________________________________________________
LSTM_3 (LSTM)                (None, 100)               142800    
_________________________________________________________________
Dense_1 (Dense)              (None, 4)                 404       
Total params: 5,169,764
Trainable params: 5,169,764
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.fit(X_train,y_train, epochs=10)

Epoch 1/10
  1/214 [..............................] - ETA: 21s - loss: 1.3836 - accuracy: 0.3750

InvalidArgumentError: ignored